In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn import tree
import xgboost as xgb

#from sklearn.metrics import accuracy_score 
#from sklearn.metrics import r2_score

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.model_selection import RandomizedSearchCV

import json

In [2]:
with open(r'gratkapl_mieszkania_sopot.json', 'r') as f:
    data2 = json.load(f)
df2 = pd.DataFrame(data2)

with open(r'gratkapl_mieszkania_gdynia.json', 'r') as f:
    data4 = json.load(f)
df4 = pd.DataFrame(data4)

with open(r'gratkapl_mieszkania_gdansk.json', 'r') as f:
    data5 = json.load(f)
df5 = pd.DataFrame(data5)


## Prepare data

In [4]:
df = pd.concat([ df2, df4, df5],sort=False, ignore_index =True)
df.shape

(3979, 48)

In [5]:
df_gratka = df[['adres','cena','cena_za_metr','data_aktualizacji','id_ogloszenia','liczba_pieter_budynku',
               'liczba_pokoi','powierzchnia', 'rok_budowy', 'tresc', 'kuchnia',
               'typ_nieruchomosci', 'tytul', 'wykonczenie', 'zdjecie','rodzaj_zabudowy', 'typ_domu', 'pietro',
                'data_dostepne', 'miejsce-parkingowe', 'forma-wlasnosci', 'oplaty-czynsz-administracyjny-media',
                 'komunikacja', 'okna'
               ]]

In [6]:
df_gratka = df_gratka.assign(adres_miasto = df_gratka['adres'].str.split(',').apply(pd.Series, 1)[0])
df_gratka['adres_miasto'].unique()

array(['Sopot', 'Gdynia', 'Gdańsk'], dtype=object)

In [7]:
df_gratka = df_gratka.assign(adres_dzielnica = df_gratka['adres'].str.split(',').apply(pd.Series, 1)[1])
df_gratka['adres_dzielnica'].unique()

array(['Dolny', 'DOLNY', 'Górny', 'Wyścigi', 'Centrum', 'GÓRNY',
       'KamiennyPotok', 'Przylesie', 'BRODWINO', 'pomorskie', 'Brodwino',
       'DolnySopot', 'Sopot', 'SopotDolny', 'SopotGórny', 'Karlikowo',
       'Świemirowo', 'GórnySopot', 'Orłowo', 'Pogórze', 'Redłowo',
       'Chylonia', 'Witomino', 'Oksywie', 'Cisowa',
       'Witomino-Leśniczówka', 'MałyKack', 'Kosakowo', 'Leszczynki',
       'DziałkiLeśne', 'Obłuże', 'KamiennaGóra', 'PustkiCisowskie',
       'Śródmieście', 'Chwarzno-Wiczlino', 'Wiczlino', 'FIKAKOWO',
       'BabieDoły', 'Grabówek', 'Dąbrowa', 'Karwiny', 'GDYNIA',
       'WielkiKack', 'CENTRUM', 'WzgórzeŚw.Maksymiliana', 'Chwarzno',
       'KaczeBuki', 'Witomino-Radiostacja', 'WzgórzeŚwiętegoMaksymiliana',
       'GdyniaChwarzno', 'StareObłuże', 'Fikakowo', 'WitominoLeśniczówka',
       'PustkiCisowskie-Demptowo', 'PustkiCiskowskie',
       'WZGÓRZEŚW.MAKSYMILIANA', 'Chyylonia', 'Wzgórześw.Maksymiliana',
       'PogórzeDolne', 'GdyniaŚródmieście', 'OruniaGórna

In [8]:
df_gratka['adres_dzielnica'].replace({'-':''},regex=True, inplace=True)
df_gratka = df_gratka.assign(adres_dzielnica = df_gratka['adres_dzielnica'].str.lower())
df_gratka['adres_dzielnica'].replace({'pomorskie': np.nan},regex=True, inplace=True)
df_gratka['adres_dzielnica'].replace({'sopot': np.nan},regex=True, inplace=True)
df_gratka['adres_dzielnica'].replace({'gdynia.': np.nan},regex=True, inplace=True)
df_gratka['adres_dzielnica'].replace({'gdańsk': np.nan},regex=True, inplace=True)
df_gratka['adres_dzielnica'].replace({'os.aniołki': 'aniołki'},regex=True, inplace=True)
df_gratka['adres_dzielnica'].replace({'wzgórześw.maksymiliana': 'wzgórzeświętegomaksymiliana'},regex=True, inplace=True)
df_gratka['adres_dzielnica'].replace({'chyylonia': 'chylonia'},regex=True, inplace=True)
df_gratka['adres_dzielnica'].replace({'wyspasobieszewska': 'sobieszewo'},regex=True, inplace=True)
df_gratka['adres_dzielnica'].unique()

array(['dolny', 'górny', 'wyścigi', 'centrum', 'kamiennypotok',
       'przylesie', 'brodwino', nan, 'karlikowo', 'świemirowo', 'orłowo',
       'pogórze', 'redłowo', 'chylonia', 'witomino', 'oksywie', 'cisowa',
       'witominoleśniczówka', 'małykack', 'kosakowo', 'leszczynki',
       'działkileśne', 'obłuże', 'kamiennagóra', 'pustkicisowskie',
       'śródmieście', 'chwarznowiczlino', 'wiczlino', 'fikakowo',
       'babiedoły', 'grabówek', 'dąbrowa', 'karwiny', 'gdynia',
       'wielkikack', 'wzgórzeświętegomaksymiliana', 'chwarzno',
       'kaczebuki', 'witominoradiostacja', 'stareobłuże',
       'pustkicisowskiedemptowo', 'pustkiciskowskie', 'pogórzedolne',
       'oruniagórna', 'wrzeszcz', 'ujeścisko', 'staremiasto', 'siedlce',
       'żabianka', 'łostowice', 'chełm', 'przymorze', 'piecki', 'jasień',
       'letnica', 'brzeźno', 'zakoniczyn', 'morena', 'przeróbka', 'oliwa',
       'suchanino', 'maćkowy', 'zaspa', 'matarnia', 'kowale', 'osowa',
       'kiełpinek', 'orunia', 'nowypo

In [9]:
del df_gratka['adres']

In [10]:
df_gratka["powierzchnia"].isnull().sum()

6

In [11]:
df_gratka["cena_za_metr"].isnull().sum()

5

In [12]:
df_gratka[(df_gratka["powierzchnia"].isnull()) & (df_gratka["cena_za_metr"]> 0)]

,cena,cena_za_metr,data_aktualizacji,id_ogloszenia,liczba_pieter_budynku,liczba_pokoi,powierzchnia,rok_budowy,tresc,kuchnia,...,typ_domu,pietro,data_dostepne,miejsce-parkingowe,forma-wlasnosci,oplaty-czynsz-administracyjny-media,komunikacja,okna,adres_miasto,adres_dzielnica
1161,970000.0,9897.96,2020-04-22,14330197,NaN,1,NaN,NaN,Numer licencji pośrednika odpowiedzialnego za ...,NaN,...,NaN,parter,NaN,NaN,NaN,NaN,NaN,NaN,Gdynia,śródmieście


In [13]:
df_gratka.loc[(df_gratka["powierzchnia"].isnull()) & (df_gratka["cena_za_metr"]> 0), "powierzchnia" ] = (df_gratka.loc[1161]['cena']/df_gratka.loc[1413]['cena_za_metr'])

In [14]:
df_gratka.shape

(3979, 25)

In [15]:
df_gratka.dropna(subset=["powierzchnia"], inplace=True)
df_gratka.shape

(3974, 25)

In [16]:
df_flats = df_gratka[(df_gratka["powierzchnia"] < 300)&(df_gratka["cena"]< 2500000)&(df_gratka["cena_za_metr"]< 25000)]
df_flats.shape

(3913, 25)

In [17]:
df_flats.head()

,cena,cena_za_metr,data_aktualizacji,id_ogloszenia,liczba_pieter_budynku,liczba_pokoi,powierzchnia,rok_budowy,tresc,kuchnia,...,typ_domu,pietro,data_dostepne,miejsce-parkingowe,forma-wlasnosci,oplaty-czynsz-administracyjny-media,komunikacja,okna,adres_miasto,adres_dzielnica
0,1990000.0,24567.90,2020-05-15,16543076,3,3,81.00,2007.0,"Apartament w doskonałej lokalizacji, dla klien...",NaN,...,blok,3,NaN,w garażu,własność,NaN,NaN,NaN,Sopot,dolny
1,1120000.0,10935.36,2020-05-14,13729739,2,4,102.42,1901.0,"ŚWIETNA LOKALIZACJA, IDEALNA INWESTYCJA, MINUT...",oddzielna,...,kamienica,2,NaN,NaN,własność,NaN,NaN,NaN,Sopot,dolny
2,667947.0,12499.01,2020-05-13,16355351,4,3,53.44,1966.0,Prezentacja bez wychodzenia z domu! Dostępny f...,NaN,...,blok,3,NaN,przynależne na ulicy,własność,500 zł,"PKP, autobus, kolej podmiejska\n",NaN,Sopot,dolny
3,1490000.0,14190.48,2020-05-14,16336719,1,4,105.00,NaN,Na sprzedaż atrakcyjna nieruchomość - lokal m...,NaN,...,dom wielorodzinny/szeregowiec,1,NaN,w garażu,własność,NaN,NaN,NaN,Sopot,górny
4,990000.0,19111.97,2020-05-12,15591724,3,2,51.80,2012.0,English version available below.\n\nLuksusowy ...,w aneksie,...,apartamentowiec,2,NaN,w garażu,własność,NaN,NaN,NaN,Sopot,wyścigi


In [18]:
df_flats_features = df_flats[['cena','cena_za_metr','liczba_pieter_budynku','liczba_pokoi','powierzchnia', 
                              'rok_budowy','kuchnia', 'wykonczenie', 'zdjecie','rodzaj_zabudowy','pietro',
                              'miejsce-parkingowe', 'forma-wlasnosci', 'oplaty-czynsz-administracyjny-media',
                              'komunikacja', 'okna','adres_dzielnica','adres_miasto'
               ]]

In [19]:
df_flats_features.head()

,cena,cena_za_metr,liczba_pieter_budynku,liczba_pokoi,powierzchnia,rok_budowy,kuchnia,wykonczenie,zdjecie,rodzaj_zabudowy,pietro,miejsce-parkingowe,forma-wlasnosci,oplaty-czynsz-administracyjny-media,komunikacja,okna,adres_dzielnica,adres_miasto
0,1990000.0,24567.90,3,3,81.00,2007.0,NaN,NaN,True,blok,3,w garażu,własność,NaN,NaN,NaN,dolny,Sopot
1,1120000.0,10935.36,2,4,102.42,1901.0,oddzielna,NaN,True,kamienica,2,NaN,własność,NaN,NaN,NaN,dolny,Sopot
2,667947.0,12499.01,4,3,53.44,1966.0,NaN,NaN,True,blok,3,przynależne na ulicy,własność,500 zł,"PKP, autobus, kolej podmiejska\n",NaN,dolny,Sopot
3,1490000.0,14190.48,1,4,105.00,NaN,NaN,NaN,True,dom wielorodzinny/szeregowiec,1,w garażu,własność,NaN,NaN,NaN,górny,Sopot
4,990000.0,19111.97,3,2,51.80,2012.0,w aneksie,NaN,True,apartamentowiec,2,w garażu,własność,NaN,NaN,NaN,wyścigi,Sopot


In [20]:
df_flats_features.shape

(3913, 18)

In [21]:
df_flats_features.isnull().sum()

cena                                      0
cena_za_metr                              0
liczba_pieter_budynku                    63
liczba_pokoi                              8
powierzchnia                              0
rok_budowy                             1182
kuchnia                                2612
wykonczenie                            3472
zdjecie                                   0
rodzaj_zabudowy                         321
pietro                                   13
miejsce-parkingowe                     1759
forma-wlasnosci                        1014
oplaty-czynsz-administracyjny-media    2809
komunikacja                            3016
okna                                   3196
adres_dzielnica                         350
adres_miasto                              0
dtype: int64

In [22]:
df_flats_features['rok_budowy'].fillna(0, inplace=True)

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\generic.py:6130: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)


In [23]:
df_flats_features['liczba_pieter_budynku'].unique()

array(['3', '2', '4', '1', '10', '9', '11', nan, '5', '6', '8', '12', '7',
       '24', '23', '28', '13', '25', '15', '17', '18', '16', '19', '14'],
      dtype=object)

In [24]:
df_flats_features['liczba_pieter_budynku'] = df_flats_features['liczba_pieter_budynku'].astype('float')
df_flats_features['liczba_pieter_budynku'].fillna(0, inplace=True)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [25]:
df_flats_features['liczba_pokoi'].unique()

array(['3', '4', '2', '5', '1', '7', '6', 'więcej niż 8', nan, '8'],
      dtype=object)

In [26]:
df_flats_features['liczba_pokoi'].replace({'więcej niż 8': '9'},regex=True, inplace=True)
df_flats_features['liczba_pokoi'] = df_flats_features['liczba_pokoi'].astype('float')

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\generic.py:6586: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [27]:
df_flats_features['liczba_pokoi'].fillna(0, inplace=True)

In [28]:
df_flats_features['pietro'].unique()

array(['3', '2', '1', '4', 'parter', '5', '6', '9', '10', nan, '11', '8',
       '7', '18', '14', '13', '12', '15', '17', '16'], dtype=object)

In [29]:
df_flats_features['pietro'].replace({'parter': '0'},regex=True, inplace=True)
df_flats_features['pietro'] = df_flats_features['pietro'].astype('float')

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [30]:
df_flats_features['pietro'].fillna(-1, inplace=True)

In [31]:
SUFFIX_CAT = '__cat'

for feat in df_flats_features.columns:
  if isinstance(df_flats_features[feat][0], list):
    continue

  factorized_values =  df_flats_features[feat].factorize()[0]
  if SUFFIX_CAT in feat:
    df_flats_features[feat] = factorized_values
  else:
    df_flats_features[feat + SUFFIX_CAT] = factorized_values

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()


In [32]:
cat_feats = [x for x in df_flats_features.columns if SUFFIX_CAT in x ]
cat_feats = [x for x in cat_feats if 'cena' not in x]
cat_feats = [x for x in cat_feats if 'powierzchnia' not in x]
cat_feats = [x for x in cat_feats if 'rok' not in x]
#cat_feats = [x for x in cat_feats if 'liczba' not in x]
#cat_feats = [x for x in cat_feats if 'pietro' not in x]
#cat_feats.append('cena_za_metr')
cat_feats.append('powierzchnia')
cat_feats.append('rok_budowy')
cat_feats.append('liczba_pieter_budynku')
cat_feats.append('liczba_pokoi')
cat_feats.append('pietro')
print(cat_feats)
len(cat_feats)

['liczba_pieter_budynku__cat', 'liczba_pokoi__cat', 'kuchnia__cat', 'wykonczenie__cat', 'zdjecie__cat', 'rodzaj_zabudowy__cat', 'pietro__cat', 'miejsce-parkingowe__cat', 'forma-wlasnosci__cat', 'oplaty-czynsz-administracyjny-media__cat', 'komunikacja__cat', 'okna__cat', 'adres_dzielnica__cat', 'adres_miasto__cat', 'powierzchnia', 'rok_budowy', 'liczba_pieter_budynku', 'liczba_pokoi', 'pietro']


19

In [33]:
df_flats_features.head()

,cena,cena_za_metr,liczba_pieter_budynku,liczba_pokoi,powierzchnia,rok_budowy,kuchnia,wykonczenie,zdjecie,rodzaj_zabudowy,...,zdjecie__cat,rodzaj_zabudowy__cat,pietro__cat,miejsce-parkingowe__cat,forma-wlasnosci__cat,oplaty-czynsz-administracyjny-media__cat,komunikacja__cat,okna__cat,adres_dzielnica__cat,adres_miasto__cat
0,1990000.0,24567.90,3.0,3.0,81.00,2007.0,NaN,NaN,True,blok,...,0,0,0,0,0,-1,-1,-1,0,0
1,1120000.0,10935.36,2.0,4.0,102.42,1901.0,oddzielna,NaN,True,kamienica,...,0,1,1,-1,0,-1,-1,-1,0,0
2,667947.0,12499.01,4.0,3.0,53.44,1966.0,NaN,NaN,True,blok,...,0,0,0,1,0,0,0,-1,0,0
3,1490000.0,14190.48,1.0,4.0,105.00,0.0,NaN,NaN,True,dom wielorodzinny/szeregowiec,...,0,2,2,0,0,-1,-1,-1,1,0
4,990000.0,19111.97,3.0,2.0,51.80,2012.0,w aneksie,NaN,True,apartamentowiec,...,0,3,1,0,0,-1,-1,-1,2,0


In [34]:
df_flats_features.isnull().sum()

cena                                           0
cena_za_metr                                   0
liczba_pieter_budynku                          0
liczba_pokoi                                   0
powierzchnia                                   0
rok_budowy                                     0
kuchnia                                     2612
wykonczenie                                 3472
zdjecie                                        0
rodzaj_zabudowy                              321
pietro                                         0
miejsce-parkingowe                          1759
forma-wlasnosci                             1014
oplaty-czynsz-administracyjny-media         2809
komunikacja                                 3016
okna                                        3196
adres_dzielnica                              350
adres_miasto                                   0
cena__cat                                      0
cena_za_metr__cat                              0
liczba_pieter_budynk

In [35]:
x = df_flats_features[cat_feats].values
y = df_flats_features[['cena']]

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.1, random_state=8)

## Choose the best features
#### test based on several models

#### Decision Tree

In [38]:
random_search = RandomizedSearchCV(tree.DecisionTreeRegressor(),
                         param_distributions ={'criterion': ["mse", "friedman_mse", "mae"],
                                                'min_samples_split': range(2, 21),
                                                'min_samples_leaf' : range(1, 21),
                                                'max_leaf_nodes' : range(2, 25), 
                                                'max_depth' : range(2, 25)
                                                },
                          cv=5)
random_search.fit(x_train, y_train)
print(random_search.best_score_)
print(random_search.best_params_)

0.7122711467340945
{'min_samples_split': 13, 'min_samples_leaf': 10, 'max_leaf_nodes': 22, 'max_depth': 6, 'criterion': 'friedman_mse'}


In [39]:
tree_reg = tree.DecisionTreeRegressor(criterion = 'friedman_mse', max_depth = 6, max_leaf_nodes = 22, 
                                      min_samples_leaf = 10, min_samples_split = 13, random_state=1)
tree_cvs = cross_val_score(tree_reg, x_train, y_train, cv=5)
tree_reg.fit(x_train, y_train)
tree_score = tree_reg.score(x_test, y_test)


print('cross_val_score:', tree_cvs)
print("Score on train data (with 95%% conf. intervals): %0.2f (+/- %0.2f)" % (tree_cvs.mean(), tree_cvs.std() * 2))
print('Score on test data:', tree_score)

cross_val_score: [0.6865368  0.73768263 0.68591128 0.72524994 0.72601164]
Score on train data (with 95% conf. intervals): 0.71 (+/- 0.04)
Score on test data: 0.5860539486696601


In [40]:
model = tree.DecisionTreeRegressor(criterion = 'friedman_mse', max_depth = 6, max_leaf_nodes = 22, 
                                      min_samples_leaf = 10, min_samples_split = 13, random_state=1)
scores = cross_val_score(model, x, y, cv=3, scoring ='neg_mean_absolute_error')
np.mean(scores)

-152146.15398724374

#### Random Forest

In [53]:
random_search = RandomizedSearchCV(RandomForestRegressor(),
                          param_distributions ={'n_estimators' : range(1,101),
                                                'criterion' : ["mse", "mae"],
                                                'max_depth' : range(2, 25)
                                                },
                          cv=5)
random_search.fit(x_train, y_train)
print(random_search.best_score_)
print(random_search.best_params_)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:514: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:514: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:514: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:514: DataConversionWarning: A column-vector y wa

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:514: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:514: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:514: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:514: DataConversionWarning: A column-vector y wa

0.8163594332431194
{'n_estimators': 100, 'max_depth': 19, 'criterion': 'mse'}


In [54]:
rf_reg = RandomForestRegressor(n_estimators =100, max_depth =19 ,criterion ='mse', random_state=1)
rf_cvs = cross_val_score(rf_reg, x_train, y_train, cv=5)
rf_reg.fit(x_train, y_train)
rf_score = rf_reg.score(x_test, y_test)

print('cross_val_score:', rf_cvs)
print("Score on train data (with 95%% conf. intervals): %0.2f (+/- %0.2f)" % (rf_cvs.mean(), rf_cvs.std() * 2))
print('Score on test data:', rf_score)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:514: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:514: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:514: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:514: DataConversionWarning: A column-vector y wa

cross_val_score: [0.81678847 0.82913661 0.80288949 0.81271906 0.81396168]
Mean accuracy (with 95% conf. intervals): 0.82 (+/- 0.02)
Score on test data: 0.7293630730053142


In [55]:
model = RandomForestRegressor(n_estimators =100, max_depth =19 ,criterion ='mse', random_state=1)
scores = cross_val_score(model, x, y, cv=3, scoring ='neg_mean_absolute_error')
np.mean(scores)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:514: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:514: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:514: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)


-132259.06306958382

#### XGBoost

In [58]:
random_search = RandomizedSearchCV(xgb.XGBRegressor(),
                          param_distributions ={'booster' : ["gbtree", "gblinear"],
                                                'max_depth' : range(2,50),
                                                'learning_rate' : np.arange(0, 1.0, 0.1),
                                                'reg_alpha': np.arange(0, 3.0, 0.1),
                                                'reg_lambda': np.arange(0, 3.0, 0.1),
                                                'n_estimators': range (1,300)
                                                },
                          cv=5)
random_search.fit(x_train, y_train)
print(random_search.best_score_)
print(random_search.best_params_)

[16:18:06] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[16:18:06] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[16:18:06] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[16:18:06] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[16:18:06] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[16:18:06] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[16:18:09] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[16:18:13] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[16:18:17] WARNING: src/objective/regression_obj.cu:152: reg:linear is n

In [59]:
xg_reg = xgb.XGBRegressor(reg_lambda= 2.7, reg_alpha= 2.4, n_estimators= 237, max_depth= 8,
                          learning_rate = 0.1, booster ='gbtree',random_state=1)
xg_cvs = cross_val_score(xg_reg, x_train, y_train, cv=5)
xg_reg.fit(x_train, y_train)
xg_score = xg_reg.score(x_test, y_test)

print('cross_val_score:', xg_cvs)
print("Score on train data (with 95%% conf. intervals): %0.2f (+/- %0.2f)" % (xg_cvs.mean(), xg_cvs.std() * 2))
print('Score on test data:', xg_score)

[16:20:08] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[16:20:10] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[16:20:11] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[16:20:12] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[16:20:14] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[16:20:15] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
cross_val_score: [0.83646301 0.841349   0.8383229  0.84117094 0.83844419]
Score on train data (with 95% conf. intervals): 0.84 (+/- 0.00)
Score on test data: 0.7483211459960712


In [60]:
model = xgb.XGBRegressor(reg_lambda= 2.7, reg_alpha= 2.4, n_estimators= 237, max_depth= 8,
                          learning_rate = 0.1, booster ='gbtree',random_state=1)
scores = cross_val_score(model, x, y, cv=3, scoring ='neg_mean_absolute_error')
np.mean(scores)

[16:20:47] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[16:20:49] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[16:20:50] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


-122972.19458312423

#### K-Nearest Neighbors

In [61]:
random_search = RandomizedSearchCV(KNeighborsRegressor(),
                          param_distributions ={'n_neighbors' : range(1, 50),
                                                'weights' : ["uniform", "distance"],
                                                'p' : [1, 2]},
                                 cv=5)
random_search.fit(x_train, y_train)
print(random_search.best_score_)
print(random_search.best_params_)

0.756482583102475
{'weights': 'distance', 'p': 1, 'n_neighbors': 8}


In [62]:
kn_reg = KNeighborsRegressor(n_neighbors = 8, p = 1, weights = 'distance')
kn_cvs = cross_val_score(kn_reg, x_train, y_train, cv=5)
kn_reg.fit(x_train, y_train)
kn_score = kn_reg.score(x_test, y_test)

print('cross_val_score:', kn_cvs)
print("Score on train data (with 95%% conf. intervals): %0.2f (+/- %0.2f)" % (kn_cvs.mean(), kn_cvs.std() * 2))
print('Score on test data:', kn_score)

cross_val_score: [0.71859276 0.75454998 0.76312491 0.77706662 0.76913246]
Score on train data (with 95% conf. intervals): 0.76 (+/- 0.04)
Score on test data: 0.7201880511649613


In [63]:
model = KNeighborsRegressor(n_neighbors = 8, p = 1, weights = 'distance')
scores = cross_val_score(model, x, y, cv=3, scoring ='neg_mean_absolute_error')
np.mean(scores)

-158406.68595710522

In [64]:
!pip install eli5

In [65]:
import eli5
from eli5.sklearn import PermutationImportance

Using TensorFlow backend.
C:\Users\Rafał\AppData\Roaming\Python\Python37\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\Rafał\AppData\Roaming\Python\Python37\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\Rafał\AppData\Roaming\Python\Python37\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\Rafał\AppData\Roa

In [66]:
m = tree.DecisionTreeRegressor(criterion = 'friedman_mse', max_depth = 6, max_leaf_nodes = 22, 
                                      min_samples_leaf = 10, min_samples_split = 13, random_state=1)
m.fit(x,y)

imp = PermutationImportance(m, random_state=0).fit(x,y)
eli5.show_weights(imp,feature_names=cat_feats)

Weight,Feature
0.9600 ± 0.0169,powierzchnia
0.3138 ± 0.0209,rodzaj_zabudowy__cat
0.0845 ± 0.0060,adres_miasto__cat
0.0218 ± 0.0081,adres_dzielnica__cat
0.0179 ± 0.0038,liczba_pieter_budynku__cat
0.0066 ± 0.0004,rok_budowy
0 ± 0.0000,pietro__cat
0 ± 0.0000,liczba_pokoi__cat
0 ± 0.0000,kuchnia__cat
0 ± 0.0000,wykonczenie__cat


In [67]:
m = RandomForestRegressor(n_estimators =100, max_depth =19 ,criterion ='mse', random_state=1)
m.fit(x,y)

imp = PermutationImportance(m, random_state=0).fit(x,y)
eli5.show_weights(imp,feature_names=cat_feats)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


Weight,Feature
1.1404 ± 0.0219,powierzchnia
0.3255 ± 0.0270,rodzaj_zabudowy__cat
0.1076 ± 0.0109,adres_dzielnica__cat
0.1005 ± 0.0044,adres_miasto__cat
0.0744 ± 0.0049,rok_budowy
0.0242 ± 0.0019,liczba_pieter_budynku__cat
0.0214 ± 0.0011,liczba_pieter_budynku
0.0188 ± 0.0014,pietro
0.0145 ± 0.0011,pietro__cat
0.0120 ± 0.0007,miejsce-parkingowe__cat


In [68]:
m = xgb.XGBRegressor(reg_lambda= 2.7, reg_alpha= 2.4, n_estimators= 237, max_depth= 8,
                          learning_rate = 0.1, booster ='gbtree',random_state=1)
m.fit(x,y)

imp = PermutationImportance(m, random_state=0).fit(x,y)
eli5.show_weights(imp,feature_names=cat_feats)

[16:30:12] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


Weight,Feature
1.0710 ± 0.0246,powierzchnia
0.1742 ± 0.0067,adres_dzielnica__cat
0.1612 ± 0.0113,rodzaj_zabudowy__cat
0.0868 ± 0.0065,rok_budowy
0.0607 ± 0.0014,adres_miasto__cat
0.0258 ± 0.0016,liczba_pieter_budynku__cat
0.0165 ± 0.0023,pietro__cat
0.0154 ± 0.0017,pietro
0.0119 ± 0.0007,liczba_pieter_budynku
0.0095 ± 0.0012,komunikacja__cat


In [69]:
m = KNeighborsRegressor(n_neighbors = 8, p = 1, weights = 'distance')
m.fit(x,y)

imp = PermutationImportance(m, random_state=0).fit(x,y)
eli5.show_weights(imp,feature_names=cat_feats)

Weight,Feature
1.0476 ± 0.0414,powierzchnia
0.3590 ± 0.0172,rok_budowy
0.3246 ± 0.0116,adres_dzielnica__cat
0.0799 ± 0.0112,oplaty-czynsz-administracyjny-media__cat
0.0584 ± 0.0055,liczba_pieter_budynku__cat
0.0508 ± 0.0053,komunikacja__cat
0.0435 ± 0.0029,liczba_pieter_budynku
0.0408 ± 0.0044,pietro__cat
0.0376 ± 0.0006,pietro
0.0310 ± 0.0014,rodzaj_zabudowy__cat


In [70]:
# THE BEST FEATURES:
cat_feats =[
'powierzchnia',
'rodzaj_zabudowy__cat',
'adres_miasto__cat',
'adres_dzielnica__cat',
'liczba_pieter_budynku__cat',
'rok_budowy',
'pietro',
'liczba_pokoi__cat'
]

## Share data 

In [71]:
df_join = df[['cena','cena_za_metr', 'powierzchnia', 'liczba_pokoi','liczba_pieter_budynku','rodzaj_zabudowy'
               ,'rok_budowy', 'adres', 'pietro', 'wykonczenie',  'kuchnia'
               ]]

In [72]:
df_join = df_join.assign(miasto = df_join['adres'].str.split(',').apply(pd.Series, 1)[0])

In [73]:
df_join = df_join.assign(dzielnica = df_join['adres'].str.split(',').apply(pd.Series, 1)[1])

In [74]:
df_join.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3979 entries, 0 to 3978
Data columns (total 13 columns):
cena                     3979 non-null float64
cena_za_metr             3974 non-null float64
powierzchnia             3973 non-null float64
liczba_pokoi             3969 non-null object
liczba_pieter_budynku    3906 non-null object
rodzaj_zabudowy          3644 non-null object
rok_budowy               2770 non-null float64
adres                    3979 non-null object
pietro                   3957 non-null object
wykonczenie              451 non-null object
kuchnia                  1321 non-null object
miasto                   3979 non-null object
dzielnica                3979 non-null object
dtypes: float64(4), object(9)
memory usage: 404.2+ KB


In [75]:
df_join.to_json(r'C:\...\gratka.json')